## 1. Segmentation Pre-processing

## 1.1 Dataset Conversion to fit the nnUNet
* original notebook: `BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet/Dataset_conversion.ipynb`

### Preparing to convert the dataset

In [1]:
import json
import shutil
import os 
import SimpleITK as sitk
import numpy as np

In [4]:
### 위치 설정
prefix_dir=".."
source_brats = f'{prefix_dir}/data/real_training_201_340' # BraTS2023 데이터  
source_gligan = f'{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/GliGAN/Checkpoint/brats2023/Synthetic_dataset_random_labels' # GliGan으로 증강한 데이터
destination_folder = f'{prefix_dir}/results/Segmentation/nnUNet/nnUNet_raw/brats2023_1' # Segmentation 데이터셋과 관련 데이터를 저장할 위치
                                                                                        # 이후 다른코드에서 작업폴더 이름을 'Dataset@@@_BraTS~'와 같이 강제하는데
                                                                                        # 폴더명을 바꾸는 코드를 따로 삽입해놓음

### 필요할 경우 destination_folder 생성하기
#os.system(f'mkdir -p {destination_folder}')

### 위치 확인
print(os.path.exists(source_brats))
print(os.path.exists(source_gligan))
print(os.path.exists(destination_folder))

True
True
True


In [ ]:
### 저장할 위치를 환경변수로 설정
#os.environ['nnUNet_preprocessed'] = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet/nnUNet_preprocessed"
#os.environ['nnUNet_results'] = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet/nnUNet_results"
#os.environ['nnUNet_raw'] = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet/nnUNet_raw"
nnUNet_preprocessed = f'{prefix_dir}/results/Segmentation/nnUNet/nnUNet_preprocessed'
nnUNet_results = f'{prefix_dir}/results/Segmentation/nnUNet/nnUNet_results'
nnUNet_raw = f'{prefix_dir}/results/Segmentation/nnUNet/nnUNet_raw'
os.environ['nnUNet_preprocessed'] = f'{nnUNet_preprocessed}'
os.environ['nnUNet_results'] = f'{nnUNet_results}'
os.environ['nnUNet_raw'] = f'{nnUNet_raw}'

### 환경변수 설정 확인
print(f"nnUNet_preprocessed: \t{os.environ['nnUNet_preprocessed']}")
print(f"nnUNet_results: \t{os.environ['nnUNet_results']}")
print(f"nnUNet_raw: \t\t{os.environ['nnUNet_raw']}")

nnUNet_preprocessed: 	../results/Segmentation/nnUNet/nnUNet_preprocessed
nnUNet_results: 	../results/Segmentation/nnUNet/nnUNet_results
nnUNet_raw: 		../results/Segmentation/nnUNet/nnUNet_raw


False

In [6]:
### BraTS 데이터의 label을 nnUNet에서 사용할 수 있게 변경하여 저장하는 함수
def copy_BraTS_segmentation_and_convert_labels_to_nnUNet_2023(in_file: str, out_file: str) -> None:
   img = sitk.ReadImage(in_file) # 이미지를 sitk 이미지로 변환
   img_npy = sitk.GetArrayFromImage(img) # sitk 이미지에서 numpy 정보 추출
   uniques = np.unique(img_npy) # 이미지를 구성하는 labels 확인 -> 0, 1, 2, 3 이어야함
   seg_new = np.zeros_like(img_npy) # img_npy와 동일한 크기의 0으로 채워진numpy를 만듦
   seg_new[img_npy == 3] = 3 
   seg_new[img_npy == 2] = 2
   seg_new[img_npy == 1] = 1
   seg_new[img_npy < 0] = 0
   seg_new[img_npy > 4] = 0
   img_corr = sitk.GetImageFromArray(seg_new)
   img_corr.CopyInformation(img)
   sitk.WriteImage(img_corr, out_file)  

In [7]:
### BraTS2023 데이터를 nnUNet에서 사용할 수 있게 변경하는 함수
def convert_to_nnUNet_2023_real(source_file, destination_folder):
    os.makedirs(os.path.join(destination_folder, "imagesTr"), exist_ok=True) # 'imagesTr' 폴더 작성하기
    os.makedirs(os.path.join(destination_folder, "labelsTr"), exist_ok=True) # 'labelsTr' 폴더 작성하기
    for folder in os.listdir(source_file):
        ### t1c 파일 복사히기
        t1c_name = f"{folder}-t1c.nii.gz" # 파일확인
        t1c_path = os.path.join(source_file, folder, t1c_name) # 저장할 경로 확인
        new_t1c_name = f"{folder}_0000.nii.gz" # 새 파일 이름은 뒤에 class를 적음. 0000은 0번 class.
        destination_t1c = os.path.join(destination_folder, "imagesTr", new_t1c_name) # 새 파일 경로 확인
        shutil.copy(t1c_path, destination_t1c) # 파일 복사하기

        ### t1n 파일 복사하기
        t1n_name = f"{folder}-t1n.nii.gz"
        t1n_path = os.path.join(source_file, folder, t1n_name)
        new_t1n_name = f"{folder}_0001.nii.gz"
        destination_t1n = os.path.join(destination_folder, "imagesTr", new_t1n_name)
        shutil.copy(t1n_path, destination_t1n)

        ### t2f 파일 복사하기        
        t2f_name = f"{folder}-t2f.nii.gz"
        t2f_path = os.path.join(source_file, folder, t2f_name)
        new_t2f_name = f"{folder}_0002.nii.gz"
        destination_t2f = os.path.join(destination_folder, "imagesTr", new_t2f_name)
        shutil.copy(t2f_path, destination_t2f)

        ### t2w 파일 복사하기        
        t2w_name = f"{folder}-t2w.nii.gz"
        t2w_path = os.path.join(source_file, folder, t2w_name)
        new_t2w_name = f"{folder}_0003.nii.gz"
        destination_t2w = os.path.join(destination_folder, "imagesTr", new_t2w_name)
        shutil.copy(t2w_path, destination_t2w)
        
        ### seg 파일 복사하기
        seg_name = f"{folder}-seg.nii.gz"
        seg_path = os.path.join(source_file, folder, seg_name)
        new_seg_name = f"{folder}.nii.gz"
        destination_seg = os.path.join(destination_folder, "labelsTr", new_seg_name)
        
        ### BraTS 데이터의 label을 nnUNet에서 사용할 수 있게 변경하여 저장
        copy_BraTS_segmentation_and_convert_labels_to_nnUNet_2023(in_file=seg_path, out_file=destination_seg) 

In [8]:
### 증강하여 만든 가상의 데이터를 nnUNet에서 사용할 수 있게 변경하는 함수
def convert_to_nnUNet_2023_fake(source_file, destination_folder):
    os.makedirs(os.path.join(destination_folder, "imagesTr"), exist_ok=True)
    os.makedirs(os.path.join(destination_folder, "labelsTr"), exist_ok=True)
    for folder in os.listdir(source_file):
        ### t1ce 파일 복사히기
        t1c_name = f"{folder}-scan_t1ce.nii.gz"
        t1c_path = os.path.join(source_file, folder, t1c_name)
        new_t1c_name = f"{folder}_0000.nii.gz"
        destination_t1c = os.path.join(destination_folder, "imagesTr", new_t1c_name)
        shutil.copy(t1c_path, destination_t1c)
        
        ### t1n 파일 복사히기
        t1n_name = f"{folder}-scan_t1.nii.gz"
        t1n_path = os.path.join(source_file, folder, t1n_name)
        new_t1n_name = f"{folder}_0001.nii.gz"
        destination_t1n = os.path.join(destination_folder, "imagesTr", new_t1n_name)
        shutil.copy(t1n_path, destination_t1n)
        
        ### flair 파일 복사히기
        t2f_name = f"{folder}-scan_flair.nii.gz"
        t2f_path = os.path.join(source_file, folder, t2f_name)
        new_t2f_name = f"{folder}_0002.nii.gz"
        destination_t2f = os.path.join(destination_folder, "imagesTr", new_t2f_name)
        shutil.copy(t2f_path, destination_t2f)
        
        ### t2 파일 복사히기
        t2w_name = f"{folder}-scan_t2.nii.gz"
        t2w_path = os.path.join(source_file, folder, t2w_name)
        new_t2w_name = f"{folder}_0003.nii.gz"
        destination_t2w = os.path.join(destination_folder, "imagesTr", new_t2w_name)
        shutil.copy(t2w_path, destination_t2w)
        
        ### seg 파일 복사히기
        seg_name = f"{folder}-seg.nii.gz"
        seg_path = os.path.join(source_file, folder, seg_name)
        new_seg_name = f"{folder}.nii.gz"
        destination_seg = os.path.join(destination_folder, "labelsTr", new_seg_name)

        ### BraTS 데이터의 label을 nnUNet에서 사용할 수 있게 변경하여 저장
        copy_BraTS_segmentation_and_convert_labels_to_nnUNet_2023(in_file=seg_path, out_file=destination_seg)


### Coverting the dataset

In [9]:
### BraTS2023 데이터 변환하기
convert_to_nnUNet_2023_real(source_brats, destination_folder) # BraTS2023 데이터를 nnUNet에서 사용할 수 있게 변경

In [10]:
### 증강하여 만든 가상의 데이터를 변환하기
convert_to_nnUNet_2023_fake(source_gligan, destination_folder) # 증강한 데이터를 nnUNet에서 사용할 수 있게 변경

## 1.2 dataset.json for nnUNet

In [11]:
### nnUNet에 필요한 json 파일 정보 
json_content = {
    "channel_names": {
        "0": "t1c",
        "1": "t1",
        "2": "t2f",
        "3": "t2"
    },
    "labels": {
        "background": 0,
        "whole tumor": [
            1,
            2,
            3
        ],
        "tumor core": [
            2,
            3
        ],
        "enhancing tumor": 3,
    },
    "numTraining": len(os.listdir(os.path.join(destination_folder, "labelsTr"))),
    "file_ending": ".nii.gz",
    "regions_class_order": [
        1,
        2,
        3
    ]
}

In [12]:
### JSON 파일 저장
with open(f'{destination_folder}/dataset.json', 'w') as json_file:
    json.dump(json_content, json_file, indent=4)

### JSON 파일 확인
os.path.exists(f'{destination_folder}/dataset.json')

True

## 1.3 nnUNetv2 Segmentation for dataset ID '232'
* 실험팀은 3개의 데이터셋에서 'k-fold cross validation'을 5번 수행하여(5-fold) 총 45개의 모델을 만들었다.
    1) dataset ID '236': Real + Synthetic data generated using real labels (3 networks * 5 folds)
    2) dataset ID '233': Real + Registered cases (3 networks * 5 folds)
    3) dataset ID '232': Real + Synthetic data generated using synthetic labels (3 networks * 5 folds)
* 실험팀은 데이터셋 ID '232'만 사용하여 만든 15개의 모델로도 충분한 성능이 나온다고 판단했다.
    * 데이터셋 ID 232의 가중치만 포함하여 [zenodo](https://zenodo.org/records/14001262)에 공개하였다.
* 우리도 232만을 대상으로 총 15개의 segmentation 모델을 만들어보고자 한다.

#### 데이터셋 ID 232를 대상으로 command 만들기

In [13]:
### 파이썬 파일 위치 확인
nnUNetv2_plan_and_preprocess = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet_install/nnunetv2/experiment_planning/plan_and_preprocess_entrypoints.py"
os.path.exists(nnUNetv2_plan_and_preprocess)

True

In [14]:
### 데이터셋 ID 설정
dataset_id = 232

### 파라미터 확인
segmentation_parameters = {
    "-d": f'{dataset_id}', # 데이터셋 ID  
    "-fpe": "DatasetFingerprintExtractor", # 사용할 데이터셋 Fingerprint Extractor 클래스 이름
    "--verify_dataset_integrity": '', # 데이터셋 무결성 확인 여부
    "-np": 8, # Fingerprint 추출에 사용할 프로세스 수.
    "--clean": '', # 기존 Fingerprint를 덮어쓸지 여부
    "--verbose": '' # 상세 로그 출력 여부
}

### 데이터셋 ID에 맞게 destination folder를 수정하기 위한 파라미터 정의
dataset_folder_name = f'Dataset{dataset_id}_BraTS_2023_rGANs'
new_destination_folder = f"{prefix_dir}/results/Segmentation/nnUNet/nnUNet_raw/{dataset_folder_name}"

In [15]:
### 데이터셋 ID에 맞게 destination folder를 수정
### 이는 dataset_name_id_conversion.py 파일에서 강제하는 규칙
os.system(f'mv {destination_folder} {new_destination_folder}')
os.path.exists(f'{new_destination_folder}')

True

In [16]:
### Pre-processingcommand 생성
cmd = f'python {nnUNetv2_plan_and_preprocess}'
for key, value in segmentation_parameters.items():
        cmd += f" {key} {value}"

### Pre-processing command 확인
cmd

'python ../BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet_install/nnunetv2/experiment_planning/plan_and_preprocess_entrypoints.py -d 232 -fpe DatasetFingerprintExtractor --verify_dataset_integrity  -np 8 --clean  --verbose '

In [17]:
### Pre-processing command 실행
os.system(cmd)

Fingerprint extraction...
Dataset232_BraTS_2023_rGANs
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
Start create_lists_from_splitted_dataset_folder
len(training_images_per_case): 493
len(training_labels_per_case): 493
Experiment planning...
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 105, 'patch_size': array([192, 160]), 'median_image_size_in_voxels': array([173., 137.]), 'spacing': array([1., 1.]), 'normalization_schemes': ['ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization'], 'use_mask_for_norm': [True, True, True, True], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_s

0

In [20]:
### Pre-processing 결과 확인
print(os.path.exists(f'{nnUNet_preprocessed}/{dataset_folder_name}'))
print(os.path.exists(f'{nnUNet_preprocessed}/{dataset_folder_name}/nnUNetPlans.json'))

True
True


### JSON 파일 준비
* 2개의 JSON 파일이 더 필요하다
    1) {nnUNet_preprocessed}/{dataset_folder_name}/nnUNetPlans.json
    2) {nnUNet_preprocessed}/{dataset_folder_name}/splits_final.json
* nnUNetPlans.json 파일 생성하는 방법
    * nnUNetPlans_2023_glioma.json 파일을 복사하고 내용을 적절하게 수정하기
        * {nnUNetv2_plan_and_preprocess}로 자동으로 생성되는 nnUNetPlans.jspn과 수치가 조금 다르다
* splits_final.json 파일 생성하는 방법 두 가지
    1) nnUNet이 자동으로 생성하게 하기
    2) 직접 Data_splits.ipynb을 이용해서 생성하기
        * 이때는 직접 BraTS 데이터를 복사해야하는데, 폴더명을 주의해서 생성하고 데이터를 복사해야한다.   
        Dataset_name_real = "Dataset231_BraTS_2023"    
        Dataset_name_fake = "Dataset232_BraTS_2023_rGANs"

In [25]:
### 복사할 nnUNetPlans_2023_glioma.json 파일 확인하기
plan_json_original = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/example/nnUNetPlans_2023_glioma.json"
os.path.exists(plan_json_original)
os.system(f'head {plan_json_original}')

{
    "dataset_name": "Dataset232_BraTS_2023_rGANs",
    "plans_name": "nnUNetPlans",
    "original_median_spacing_after_transp": [
        1.0,
        1.0,
        1.0
    ],
    "original_median_shape_after_transp": [
        139,


0

In [ ]:
### nnUNetPlans.json을 복사하기
plan_json = f'{nnUNet_preprocessed}/{dataset_folder_name}/nnUNetPlans.json'
os.system(f'cp {plan_json_original} {plan_json}')

{
    "dataset_name": "Dataset232_BraTS_2023_rGANs",
    "plans_name": "nnUNetPlans",
    "original_median_spacing_after_transp": [
        1.0,
        1.0,
        1.0
    ],
    "original_median_shape_after_transp": [
        139,


0

In [ ]:
### nnUNetPlans.json 내용 수정하기
os.system(f"sed -i 's/Dataset232_BraTS_2023_rGANs/{dataset_folder_name}/g' {plan_json}")
os.system(f'head {plan_json}')

{
    "dataset_name": "Dataset232_BraTS_2023_rGANs",
    "plans_name": "nnUNetPlans",
    "original_median_spacing_after_transp": [
        1.0,
        1.0,
        1.0
    ],
    "original_median_shape_after_transp": [
        139,
        171,
        136
    ],
    "image_reader_writer": "SimpleITKIO",
    "transpose_forward": [


0

In [ ]:
### splits_final.json(data_split.json) 파일은 nnUNEt이 자동으로 생성하게 만들기로 결정.

# 2. Segmentation Training

In [27]:
### 파이썬 위치 확인
nnUNetv2_train = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet_install/nnunetv2/run/run_training.py"
os.path.exists(nnUNetv2_train)

True

In [28]:
### nnUNetTrainer.py의 파라미터가 segmentation model training에 영향을 준다.
nnUNetTrainer = f'{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet_install/nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py'
os.path.exists(nnUNetTrainer)

True

In [29]:
### nnUNetv2_train에 '-tr nnUNetTrainer_1epoch' 옵션을 추가하여 epoch을 조정할 수도 있지만, 깃허브에서 라이브러리를 수동으로 추가해야한다.
### 따라서 nnUNetTrainer.py의 파라미터를 하드코딩하되, 백업을 생성하고 원본을 수정하는 것을 반복한다.
### 파라미터
modified_parameters ={
        'num_epochs = 1000' : 'num_epochs = 1', # modified
        }

In [31]:
### k-fold nnUNet을 5번 수행
for fold in range(5):
    print(f"=====FOLD: {fold}  start=====")
    ### 백업 생성
    os.system(f'cp {nnUNetTrainer} {nnUNetTrainer}.backup') 

    ### 파라미터 수정
    for params, val in modified_parameters.items():
        os.system(f'sed -i "s/{params}/{val}/g" {nnUNetTrainer}')

    ### nnUNet segmentation model training
    os.system(f'python {nnUNetv2_train} {dataset_id} 3d_fullres {fold} -device cuda --npz --c')
    
    ### Swin UNETR
    os.system(f'python {nnUNetv2_train} {dataset_id} 3d_fullres {fold} -device cuda --npz --c -tr nnUNetTrainer_SwinUNETR')
    
    ### 2021 Winner
    os.system(f'python {nnUNetv2_train} {dataset_id} 3d_fullres {fold} -device cuda --npz --c -tr nnUNetTrainerBN_BS5_RBT_DS_BD_PS')
    
    ### 백업 복원
    os.system(f'mv {nnUNetTrainer}.backup {nnUNetTrainer}')
    print(f"=====FOLD: {fold}  end=====")

=====FOLD: 0  start=====
Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



/home/ec2-user/project/pre/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet_install/nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py:156: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.grad_scaler = GradScaler() if self.device.type == 'cuda' else None
/home/ec2-user/miniconda/envs/BraTS_solutions/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Using DC_and_BCE_loss

This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 5, 'patch_size': [128, 160, 112], 'median_image_size_in_voxels': [139.0, 171.0, 136.0], 'spacing': [1.0, 1.0, 1.0], 'normalization_schemes': ['ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization'], 'use_mask_for_norm': [True, True, True, True], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': [2, 2, 2, 2, 2, 2], 'n_conv_per_stage_decoder': [2, 2, 2, 2, 2], 'num_pool_per_axis': [5, 5, 4], 'pool_op_kernel_sizes': [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 1]], 'conv_kernel_sizes': [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]], 'unet_max_num_features': 320, 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling_fn_seg': 'resample_data_or_seg_to_shape',

/home/ec2-user/project/pre/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet_install/nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py:156: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.grad_scaler = GradScaler() if self.device.type == 'cuda' else None
/home/ec2-user/miniconda/envs/BraTS_solutions/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Using DC_and_BCE_loss

This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 5, 'patch_size': [128, 160, 112], 'median_image_size_in_voxels': [139.0, 171.0, 136.0], 'spacing': [1.0, 1.0, 1.0], 'normalization_schemes': ['ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization'], 'use_mask_for_norm': [True, True, True, True], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': [2, 2, 2, 2, 2, 2], 'n_conv_per_stage_decoder': [2, 2, 2, 2, 2], 'num_pool_per_axis': [5, 5, 4], 'pool_op_kernel_sizes': [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 1]], 'conv_kernel_sizes': [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]], 'unet_max_num_features': 320, 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling_fn_seg': 'resample_data_or_seg_to_shape',

...

Using DC_and_BCE_loss

This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 5, 'patch_size': [128, 160, 112], 'median_image_size_in_voxels': [139.0, 171.0, 136.0], 'spacing': [1.0, 1.0, 1.0], 'normalization_schemes': ['ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization'], 'use_mask_for_norm': [True, True, True, True], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': [2, 2, 2, 2, 2, 2], 'n_conv_per_stage_decoder': [2, 2, 2, 2, 2], 'num_pool_per_axis': [5, 5, 4], 'pool_op_kernel_sizes': [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 1]], 'conv_kernel_sizes': [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]], 'unet_max_num_features': 320, 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling_fn_seg': 'resample_data_or_seg_to_shape',

/home/ec2-user/miniconda/envs/BraTS_solutions/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Traceback (most recent call last):
  File "/home/ec2-user/project/pre/task1_script_kor/../BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet_install/nnunetv2/run/run_training.py", line 259, in <module>
    run_training_entry()
  File "/home/ec2-user/project/pre/task1_script_kor/../BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet_install/nnunetv2/run/run_training.py", line 253, in run_training_entry
    run_training(args.dataset_name_or_id, args.configuration, args.fold, args.tr, args

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

USING nnUNetTrainerBN_BS5_RBT_DS_BD_PS
num_stages: 6
dim: 3


/home/ec2-user/project/pre/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet_install/nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py:156: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.grad_scaler = GradScaler() if self.device.type == 'cuda' else None
/home/ec2-user/miniconda/envs/BraTS_solutions/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Using DC_and_BCE_loss
deep_supervision_scales: [[np.float64(1.0), np.float64(1.0), np.float64(1.0)], [np.float64(0.5), np.float64(0.5), np.float64(0.5)], [np.float64(0.25), np.float64(0.25), np.float64(0.25)], [np.float64(0.125), np.float64(0.125), np.float64(0.125)], [np.float64(0.0625), np.float64(0.0625), np.float64(0.0625)]]

This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 5, 'patch_size': [128, 160, 112], 'median_image_size_in_voxels': [139.0, 171.0, 136.0], 'spacing': [1.0, 1.0, 1.0], 'normalization_schemes': ['ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization'], 'use_mask_for_norm': [True, True, True, True], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': [2, 2, 2, 2, 2, 2], 'n_conv_per_stage_decoder': [2, 2, 2, 2, 2], 'num_pool_per_axis': [5, 5, 4], 'pool_op_k

In [ ]:
### 0~5번 중 0번 fold nnUNet 수행
